In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import datetime as datetime
import os
from pathlib import Path
import requests

In [2]:
try:
  espcul_gdf = gpd.read_file('./inputs/origenes.shp')

except Exception as e:
  espcul_gdf = gpd.read_file('./inputs/origenes.gpkg')

espcul_gdf = espcul_gdf.loc[espcul_gdf.campania == '24/25']

espcul_gdf = espcul_gdf[['zona', 'idest', 'geometry']]

espcul_gdf = espcul_gdf.loc[espcul_gdf.zona.isin(['BA OESTE DX'])]

espcul_gdf = gpd.GeoDataFrame(espcul_gdf, geometry='geometry').reset_index()
espcul_gdf['geometry'] = espcul_gdf.buffer(0.0001)
espcul_dissolve = espcul_gdf.dissolve(by='idest').reset_index()

print(f"espcul_dissolve.shape: {espcul_dissolve.shape}")
print(f"espcul_gdf.shape: {espcul_gdf.shape}")
espcul_dissolve.head()

/var/folders/xt/0fzdrsd52bj4z2cqvy77d5w00000gn/T/ipykernel_15887/1350952993.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espcul_gdf['geometry'] = espcul_gdf.buffer(0.0001)


espcul_dissolve.shape: (23, 4)
espcul_gdf.shape: (190, 4)


,idest,geometry,index,zona
0,Chapaico,"MULTIPOLYGON (((-62.045 -36.70945, -62.045 -36...",42,BA OESTE DX
1,Don Justo,"MULTIPOLYGON (((-61.93264 -36.40939, -61.93264...",389,BA OESTE DX
2,El Capricho,"MULTIPOLYGON (((-61.88462 -36.3919, -61.88462 ...",393,BA OESTE DX
3,El Silencio,"MULTIPOLYGON (((-62.25912 -36.68467, -62.25913...",71,BA OESTE DX
4,El Taita,"POLYGON ((-62.24397 -36.68029, -62.24397 -36.6...",1198,BA OESTE DX


In [3]:
# origen_zona = {"O1": "BA SDE S", "O2": "BA SDE S"}
origen_zona = {}
for i in range(len(espcul_gdf)):
    origen_zona[espcul_gdf['idest'][i]] = espcul_gdf['zona'][i]

In [4]:
zonas_list = espcul_gdf.zona.unique()
zonas_list

array(['BA OESTE DX'], dtype=object)

In [5]:
espcul_dissolve['x']= espcul_dissolve['geometry'].centroid.x
espcul_dissolve['y']= espcul_dissolve['geometry'].centroid.y
espcul_dissolve['centres']= espcul_dissolve['geometry'].centroid
espcul_centroids = espcul_dissolve.loc[:,['idest','x','y','centres']].copy()
espcul_centroids = espcul_centroids.rename(columns = {'centres': 'geometry'})
espcul_centroids = espcul_centroids.to_crs('EPSG:4326')
print(f"espcul_centroids: {espcul_centroids.shape[0]}")
espcul_centroids.head()

espcul_centroids: 23


/var/folders/xt/0fzdrsd52bj4z2cqvy77d5w00000gn/T/ipykernel_15887/1240167303.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espcul_dissolve['x']= espcul_dissolve['geometry'].centroid.x
/var/folders/xt/0fzdrsd52bj4z2cqvy77d5w00000gn/T/ipykernel_15887/1240167303.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espcul_dissolve['y']= espcul_dissolve['geometry'].centroid.y
/var/folders/xt/0fzdrsd52bj4z2cqvy77d5w00000gn/T/ipykernel_15887/1240167303.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espcul_dissolve['centres']= espcul_dissolve['geometry'].centroid


,idest,x,y,geometry
0,Chapaico,-62.030718,-36.695847,POINT (-62.03072 -36.69585)
1,Don Justo,-61.917155,-36.404137,POINT (-61.91715 -36.40414)
2,El Capricho,-61.879743,-36.388696,POINT (-61.87974 -36.3887)
3,El Silencio,-62.242216,-36.672161,POINT (-62.24222 -36.67216)
4,El Taita,-62.229078,-36.677755,POINT (-62.22908 -36.67776)


In [6]:
espcul_centroids.to_file(driver = "GPKG",filename='./inputs/centoids.gpkg', encoding='utf-8', index=False)

In [7]:
espcul_coord = {}

for i in range(0, len(espcul_centroids)):
    origin = espcul_centroids['idest'][i]
    x = espcul_centroids['geometry'][i].x
    y = espcul_centroids['geometry'][i].y
    coords = (y, x)
    espcul_coord[origin]=coords
# espcul_coord

In [8]:
try:
  destinos_gdf = gpd.read_file('./inputs/destinos.shp')

except Exception as e:
  destinos_gdf = gpd.read_file('./inputs/destinos.gpkg')


print(destinos_gdf.shape)
destinos_gdf.head()

destinos_coord = {}

for i in range(len(destinos_gdf)):
    destino = destinos_gdf['Localidad'][i]
    punto = destinos_gdf['geometry'][i].centroid
    coords = (punto.y, punto.x)
    destinos_coord[destino] = coords

list(destinos_coord.items())[:4]

(69, 12)


[('9 de Julio', (-35.44393528, -60.88462748)),
 ('Arrecifes', (-34.06751491, -60.10869159)),
 ('Azul', (-36.777447, -59.86344305)),
 ('Bahia Blanca', (-38.71760509, -62.26544693))]

The centroids may be away form actual street network. Use osmnx to find the closest node on OSM network for routing

In [9]:
arr = espcul_gdf.to_crs('EPSG:4326').total_bounds
tupla = tuple(arr)
# print(tupla)

In [10]:
G = ox.graph.graph_from_bbox(tupla, network_type='drive')
nodes = G.nodes()

In [11]:
espcul_coord_snapped = {}
for name, coords in espcul_coord.items():
    node = ox.distance.nearest_nodes(G, coords[1], coords[0])
    info = nodes[node]
    espcul_coord_snapped[name] = (info['y'], info['x'])

list(espcul_coord_snapped.items())[:4]

ImportError: scikit-learn must be installed as an optional dependency to search an unprojected graph.

In [ ]:
service = 'table'
version = 'v1'
profile = 'driving'

In [ ]:
# Build list of all coordinates (espcul_coord_snapped first, then destinos_coord)
all_coords = []
# To keep track of index mapping
origin_keys = list(espcul_coord_snapped.keys())
dest_keys = list(destinos_coord.keys())

for key in origin_keys:
    o = espcul_coord_snapped[key]
    # OSRM expects lon,lat; our tuples are (lat, lon) so index [1] is lon, [0] is lat
    all_coords.append(f"{o[1]},{o[0]}")

for key in dest_keys:
    d = destinos_coord[key]
    # Same format: (lat, lon) so we use [1] for lon, [0] for lat
    all_coords.append(f"{d[1]},{d[0]}")

# Build the coordinate string
coords_str = ";".join(all_coords)

# Build sources indices: since espcul_coord_snapped are first in the list, their indices = 0..len(espcul_coord_snapped)-1
sources_indices = ";".join(str(i) for i in range(len(origin_keys)))

# Build destination indices: destinos_coord start at index len(espcul_coord_snapped)
dest_start = len(origin_keys)
dest_indices = ";".join(str(dest_start + i) for i in range(len(dest_keys)))

# Build the request URL
profile = "driving"  # or "bike"/"foot" according to your case
port = 5001  # local OSRM server port
base_url = f"http://localhost:{port}"  # local OSRM server
url = (
    f"{base_url}/table/v1/{profile}/{coords_str}"
    f"?sources={sources_indices}&destinations={dest_indices}&annotations=distance"
)

# Send the request
resp = requests.get(url)
resp.raise_for_status()
data = resp.json()

# Check for successful code
if data.get("code") != "Ok":
    raise Exception(f"OSRM table error: {data.get('code')}")

# Extract distances matrix (in meters)
distances = data.get("distances")

# Map results to list of dicts with origen, destino, distancia Y COORDENADAS
results = []
for i, orig_key in enumerate(origin_keys):
    orig_coord = espcul_coord_snapped[orig_key]
    for j, dest_key in enumerate(dest_keys):
        dest_coord = destinos_coord[dest_key]
        # Convert from meters to kilometers
        results.append({
            'origen': orig_key,
            'origen_lat': orig_coord[0],
            'origen_lon': orig_coord[1],
            'destino': dest_key,
            'destino_lat': dest_coord[0],
            'destino_lon': dest_coord[1],
            'distancia': distances[i][j] / 1000
        })

print(results)

In [ ]:
result_df = pd.DataFrame(results)
result_df.distancia = result_df.distancia.round(2)
result_df.head()

In [ ]:
path = "./outputs"

if not os.path.exists(path):
    os.makedirs(path)

# df.to_csv('matrix.csv', index=False)
result_df.to_excel('./outputs/matriz_de_distancias.xlsx', index=False)